<a href="https://colab.research.google.com/github/MateosCz/CirculantNUTS/blob/main/circulant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch
!pip3 install pyro-ppl
!pip3 install arviz
!pip3 install matplotlib
!pip3 install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 727 kB 25.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch                                                                                           
import pyro
import arviz
import pyro.distributions as pdist
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from torch.fft import irfft, rfft, fft, ifft
from pyro.infer.mcmc import HMC, MCMC, NUTS
from pyro.infer.abstract_infer import TracePredictive
from pyro.nn.module import PyroModule, to_pyro_module_, PyroSample

In [ ]:
# Check pyro version
# assert pyro.__version__.startswith('1.8.2')
# Set seed for reproducibility
pyro.set_rng_seed(1)

In [ ]:
def get_mnist(n, m):
    """
    Download MNIST and return train and evaluation sets.
    """
    img_to_tensor = transforms.ToTensor()
    mnist = datasets.MNIST('data',
        train=True,
        download=True,
        transform=img_to_tensor)
    mnist = list(mnist)
    # One batch with all of mnist
    train_loader = torch.utils.data.DataLoader(mnist,
        batch_size=len(mnist),
        shuffle=True)
    # x = images tensor, y = labels tensor 
    x, y = list(train_loader)[0]
    # Flatten images
    x = x.view(-1, 28*28)
    # Normalize
    x = (x - x.mean()) / x.std()
    # Train and test set
    train_x, train_y = x[0:n], y[0:n]
    val_x, val_y = x[n:n+m], y[n:n+m]
    return train_x, train_y, val_x, val_y


In [ ]:
def accuracy(pred, data):
  """
  Calculate accuracy of predicted labels (integers).

  pred: predictions, tensor[sample_index, chain_index, data_index, logits]
  data: actual data (digit), tensor[data_index]

  Prediction is taken as most common predicted value.
  Returns accuracy (#correct/#total).
  """
  n=data.shape[0]
  correct=0
  total=0
  for i in range(0, n):
      # Get most common prediction value from logits
      pred_i=int(torch.argmax(torch.sum(pred[:,0,i,:],0)))
      # Compare prediction with data
      if int(data[i])==int(pred_i):
          correct+=1.0
      total+=1.0
  # Return fractional accuracy
  return correct/total


In [ ]:
# MNIST has 60000 images
_x, _y, _xv, _yv = get_mnist(50000,10000)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
class PyroConv(PyroModule):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Conv2d(
                in_channels=1,
                out_channels=8,
                kernel_size=5,
                stride=1,
                padding=2,
                bias = False
            )
        # Lift to Pyro
        to_pyro_module_(self.conv)
        # 2 x 1 5 x 5 (50 rvs)
        conv_shape = self.conv.weight.shape
        # Put Normal prior on conv weights
        self.conv.weight = PyroSample(prior = pdist.Normal(0, 1).expand(conv_shape).to_event(2))
        self.maxp = torch.nn.MaxPool2d(2)

    def forward(self, x):
        # Out: N x 2 x 28 x 28
        cx = self.conv(x)
        px = self.maxp(cx)
        # Return N x 1568
        return torch.flatten(px, 1)


In [ ]:
def circ_matmul(x, w):
    # xw = fft(fft(w)*ifft(x)).real
    # Note the use of the n argument to get right output shape
    xw = irfft(torch.conj(rfft(w)) * rfft(x), n=w.shape[0])
    return xw


Customizable stuff

In [ ]:
# Training
N=200
# Test
M=200

# Get the training and test data from the MNIST global variables
x, y, xv, yv = _x[0:N], _y[0:N], _xv[N:N+M], _yv[N:M+M]

# non = torch.tanh
# non = torch.nn.ReLU()
# non = m = torch.nn.LeakyReLU(0.1)
# non = torch.nn.SiLU()
non = torch.nn.Softplus()
dim1 = 128
dim2 = 32
d1 = pdist.Bernoulli(torch.tensor(0.5)).expand([2*28*28]).sample()
d2 = pdist.Bernoulli(torch.tensor(0.5)).expand([dim1]).sample()
d1 = 2*d1 - 1
d2 = 2*d2 - 1
conv = PyroConv()

print("Parameters: ", 200 + 2*28*28 + dim1 + dim1 + dim2 + dim2*10 + 10)
print("Full: ", 200 + 1568*dim1+dim1 + dim1*dim2+dim2 +dim2*10+10)


Parameters:  2386
Full:  205490


In [ ]:
def model(x, y=None, dim1 = dim1, dim2 = dim2):
    w1 = pyro.sample("w1", pdist.Normal(0,1).expand([2*28*28]).to_event(1))
    b1 = pyro.sample("b1", pdist.Normal(0,1).expand([dim1]).to_event(1))

    w2 = pyro.sample("w2", pdist.Normal(0,1).expand([dim1]).to_event(1))
    b2 = pyro.sample("b2", pdist.Normal(0,1).expand([dim2]).to_event(1))

    w3 = pyro.sample("w3", pdist.Normal(0,1).expand([dim2,10]).to_event(2))
    b3 = pyro.sample("b3", pdist.Normal(0,1).expand([10]).to_event(1))

    # Convolution
    cx = non(conv(x.view((-1,1,28,28))))

    # Layer 1: dim1
    h1 = circ_matmul(cx*d1, w1)
    h1 = non(h1[:, 0:dim1] + b1)
    # Layer 2: dim2
    h2 = circ_matmul(h1*d2, w2)
    h2 = non(h2[:, 0:dim2] + b2)
    # Layer 3: dim=10 (logits)
    h3 = h2@w3 + b3
    # Register the logits for easy prediction
    pyro.deterministic("logits", h3)

    # Likelihood
    with pyro.plate("labels", x.shape[0]):
        y_obs = pyro.sample("y_obs", pdist.Categorical(logits = h3), obs = y)


In [ ]:
# Default max_tree_depth is 10
kernel = NUTS(model,
              jit_compile = False,
              target_accept_prob = 0.8,
              max_tree_depth = 10)

mcmc = MCMC(kernel,
            num_samples = 100,
            warmup_steps = 50,
            num_chains = 2,
            disable_progbar = False)

mcmc.run(x, y)


/usr/local/lib/python3.8/dist-packages/pyro/infer/mcmc/api.py:497: UserWarning: num_chains=2 is more than available_cpu=1. Chains will be drawn sequentially.
  warnings.warn(
Sample [0]:  44%|████▍     | 66/150 [25:20, 38.69s/it, step size=1.30e-03, acc. prob=0.888]

In [ ]:
posterior_samples = mcmc.get_samples()

posterior_predictive_test = pyro.infer.Predictive(model, posterior_samples)(
        xv, None)

posterior_predictive_train = pyro.infer.Predictive(model, posterior_samples)(
        x, None)

prior_predictive = pyro.infer.Predictive(model, num_samples=500)(
        xv, None)


In [ ]:
# Diagnostics from Arviz
data = arviz.from_pyro(mcmc)
summary = arviz.summary(data)
print(summary)

# Diagnostics from Pyro
report = mcmc.diagnostics()

# Accuracy on test set
logits = posterior_predictive_test['logits']
print("Success posterior test = %.3f" % accuracy(logits, yv))

# Accuracy on training set
logits = posterior_predictive_train['logits']
print("Success posterior training = %.3f" % accuracy(logits, y))

logits = prior_predictive['logits']
print("Success prior = %.3f" % accuracy(logits, yv))